In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time

def createStates(stationCapacity, maxArrivals):
    S = []
    V_t1 = {}
    V_t = {}
    newPolicy = {}
    oldPolicy = {}
    for state1 in range(0, stationCapacity + 1):
        for state2 in range(0, stationCapacity + 1):
            for state3 in range(0, stationCapacity + 1):
                for state4 in range(0, stationCapacity + 1):
                    for state5 in range(0, stationCapacity + 1):
                        S.append([state1, state2, state3, state4, state5])
                        V_t1[(state1, state2, state3, state4, state5)] = 0
                        V_t[(state1, state2, state3, state4, state5)] = 0
                        newPolicy[(state1, state2, state3, state4, state5)] = random.randint(0,1)
                        oldPolicy[(state1, state2, state3, state4, state5)] = random.randint(0,1)
                        
    
    E = []
    for env1 in range(1, maxArrivals + 1):
        for env2 in range(1, maxArrivals + 1):
            for env3 in range(1, maxArrivals + 1):
                for env4 in range(1, maxArrivals + 1):
                    for env5 in range(1, maxArrivals + 1):
                        E.append([env1, env2, env3, env4, env5])
    return S, V_t1, V_t, newPolicy, oldPolicy, E

def getReward(c_f, c_h, a, s, K, stationCapacity):
    return a * c_f + sum(getNextState(a, s, K, stationCapacity) * c_h)

def getNextState(a, s, K, stationCapacity, env=np.array([0,0,0,0,0])):
    s=np.clip(np.add(s, env), 0, stationCapacity)
    s_prime = s.copy()
    if a == 1:
        for i in range(len(s_prime)-1, -1, -1):
            s_prime[i] = max(s_prime[i] - K, 0)
            K -= np.abs(s[i] - s_prime[i])
    return s_prime

def getExpectedValue(environments, state, S, action, shuttleCapacity, stationCapacity, V_prime, reward, discount):
    value = 0
    for env in environments:
        s_prime = getNextState(action, state, shuttleCapacity, stationCapacity, env)
        # calculate the expectation given the uniform distribution of five possibilities
        value += 1/len(E) * V_prime[tuple(s_prime)]
    # expected total value given the state and the action
    return reward + discount * value

In [2]:
def policyIteration(theta, delta, S, A, E, K, stationCapacity, c_f, c_h, d, V_k1, V_k, oldPolicy, newPolicy):
    changing = True
    while changing:
        i = 0
        # Policy evaluation
        while delta > theta:
            delta = 0
            for s in S:
                # Get the action based on the policy
                a = oldPolicy[tuple(s)]
                reward = getReward(c_f, c_h, a, s, K, stationCapacity)
                V_k1[tuple(s)] = getExpectedValue(E, s, S, a, K, stationCapacity, V_k, reward, d)
                if np.abs(V_k1[tuple(s)] - V_k[tuple(s)]) > delta:
                    delta = np.abs(V_k1[tuple(s)] - V_k[tuple(s)])
            print(i, delta)
            V_k = V_k1.copy()
            i += 1
        # Policy improvement
        for s in S:
            expectedValues = []
            for a in A:
                reward = getReward(c_f, c_h, a, s, K, stationCapacity)
                expectedValues.append(getExpectedValue(E, s, a, K, stationCapacity, V_k, reward, d))
            bestAction = np.argmax(expectedValues)
            newPolicy[tuple(s)] = bestAction
        if newPolicy != oldPolicy:
            oldPolicy = newPolicy.copy()
            delta = 2
        else:
            changing = False
    return newPolicy

In [3]:
stationCapacity = 2
maxArrivals = 5
start = time.time()
theta = 10e-6
delta = 2
A = [0,1]
K = 30
c_f = -100
c_h = np.array([-1, -1.5, -2, -2.5, -3])
d = 0.95
S, V_k1, V_k, newPolicy, oldPolicy, E = createStates(stationCapacity, maxArrivals)
policy = policyIteration(theta, delta, S, A, E, K, stationCapacity, c_f, c_h, d, V_k1, V_k, oldPolicy, newPolicy)
end = time.time()
end - start

0 100.0
1 95.0
2 90.25


KeyboardInterrupt: 